In [ ]:
import sys

sys.path.append("../")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rumboost.rumboost import rum_train
from rumboost.datasets import load_preprocess_LPMC
from rumboost.metrics import cross_entropy

import lightgbm
import hyperopt
import numpy as np


# Example: Nested logit model (correlation amongst alternatives)

This notebook shows features implemented in RUMBoost through an example on the LPMC dataset, a mode choice dataset in London developed Hillel et al. (2018). You can find the original source of data [here](https://www.icevirtuallibrary.com/doi/suppl/10.1680/jsmic.17.00018) and the original paper [here](https://www.icevirtuallibrary.com/doi/full/10.1680/jsmic.17.00018).

We first load the preprocessed dataset and its folds for cross-validation. You can find the data under the Data folder

In [ ]:
#load dataset
LPMC_train, LPMC_test, folds = load_preprocess_LPMC(path="../Data/")

## Nested Logit model

We relax the assumption that the error term is distributed i.i.d.. If we assume that alternatives are correlated, we obtain a nested logit-like model. Nested logit probabilities are implemented in RUMBoost. The additional parameter, the scale of a nest $\mu$, can be estimated with two ways:
1. by a hyperparameter search
2. optimised within the trianing loop

Training a nested logit-like rumboost model requires an additional dictionary in the model specification dictionary. The nested logit disctionary follows the following form:

- ```mu```: a list containing the values (as float) of mu for each nest, e.g. ```[mu_nest_0, mu_nest_1]```
- ```nests```: a dictionary representing the nesting structure. Keys are the nests id, and values are the the list of alternatives in the corresponding nest. For example {0: [0, 1], 1: [2, 3]} means that alternative 0 and 1 are in nest 0, and alternative 2 and 3 are in nest 1.
- `optimise_mu`: a boolean or list of boolean. If it is a simple boolean and True, all mu values are found through scipy.minimize. If it is a list of boolean, it should be the same size than `mu` and it represents which value should be optimised or not.
  
In this example, we assume that PT and car are in a 'motorised' nest, while the walking and cycling alternative are in their own nests.

### General parameters

You can find an example of general parameters below. Unless stated otherwise, the parameters are the same than in LightGBM, since these parameters are applied directly to LightGBM Booster objects. You can find more information in the LightGBM [docs](https://lightgbm.readthedocs.io/en/stable/Parameters.html#).  For a simple RUMBoost, we recommend letting most of the parameters with default values, as RUMBoost is less sensitive to overfitting. **For a multiclass classification problem, you need to specify the num_classes parameter with the appropriate number of classes**.

In [ ]:
# parameters
general_params = {
    "n_jobs": -1,
    "num_classes": 4,  # important
    "verbosity": 1,  # specific RUMBoost parameter
    "num_iterations": 3000,
    "early_stopping_round": 100,
}

### Random Utility Model structure


In [ ]:
rum_structure = [
    {
        "utility": [0],
        "variables": [
            "age",
            "female",
            "day_of_week",
            "start_time_linear",
            "car_ownership",
            "driving_license",
            "purpose_B",
            "purpose_HBE",
            "purpose_HBO",
            "purpose_HBW",
            "purpose_NHBO",
            "fueltype_Average",
            "fueltype_Diesel",
            "fueltype_Hybrid",
            "fueltype_Petrol",
            "distance",
            "dur_walking",
        ],
        "boosting_params": {
            "monotone_constraints_method": "advanced",
            "max_depth": 1,
            "n_jobs": -1,
            "learning_rate": 0.1,
            "monotone_constraints": [
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                -1,
                -1,
            ],
            "interaction_constraints": [
                [0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
            ],
        },
        "shared": False,
    },
    {
        "utility": [1],
        "variables": [
            "age",
            "female",
            "day_of_week",
            "start_time_linear",
            "car_ownership",
            "driving_license",
            "purpose_B",
            "purpose_HBE",
            "purpose_HBO",
            "purpose_HBW",
            "purpose_NHBO",
            "fueltype_Average",
            "fueltype_Diesel",
            "fueltype_Hybrid",
            "fueltype_Petrol",
            "distance",
            "dur_cycling",
        ],
        "boosting_params": {
            "monotone_constraints_method": "advanced",
            "max_depth": 1,
            "n_jobs": -1,
            "learning_rate": 0.1,
            "monotone_constraints": [
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                -1,
                -1,
            ],
            "interaction_constraints": [
                [0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
            ],
        },
        "shared": False,
    },
    {
        "utility": [2],
        "variables": [
            "age",
            "female",
            "day_of_week",
            "start_time_linear",
            "car_ownership",
            "driving_license",
            "purpose_B",
            "purpose_HBE",
            "purpose_HBO",
            "purpose_HBW",
            "purpose_NHBO",
            "fueltype_Average",
            "fueltype_Diesel",
            "fueltype_Hybrid",
            "fueltype_Petrol",
            "distance",
            "dur_pt_access",
            "dur_pt_bus",
            "dur_pt_rail",
            "dur_pt_int_waiting",
            "dur_pt_int_walking",
            "pt_n_interchanges",
            "cost_transit",
        ],
        "boosting_params": {
            "monotone_constraints_method": "advanced",
            "max_depth": 1,
            "n_jobs": -1,
            "learning_rate": 0.1,
            "monotone_constraints": [
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                -1,
                -1,
                -1,
                -1,
                -1,
                -1,
                -1,
                -1,
            ],
            "interaction_constraints": [
                [0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
            ],
        },
        "shared": False,
    },
    {
        "utility": [3],
        "variables": [
            "age",
            "female",
            "day_of_week",
            "start_time_linear",
            "car_ownership",
            "driving_license",
            "purpose_B",
            "purpose_HBE",
            "purpose_HBO",
            "purpose_HBW",
            "purpose_NHBO",
            "fueltype_Average",
            "fueltype_Diesel",
            "fueltype_Hybrid",
            "fueltype_Petrol",
            "distance",
            "dur_driving",
            "cost_driving_fuel",
            "congestion_charge",
            "driving_traffic_percent",
        ],
        "boosting_params": {
            "monotone_constraints_method": "advanced",
            "max_depth": 1,
            "n_jobs": -1,
            "learning_rate": 0.1,
            "monotone_constraints": [
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                -1,
                -1,
                -1,
                -1,
                -1,
            ],
            "interaction_constraints": [
                [0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
            ],
        },
        "shared": False,
    },
]

### $\mu$ hyperparameter search

We treat $\mu$ as a hyperparameter. We use hyperopt to find the optimal value of the hyperparameter. More details on how to use hyperopt [here](https://hyperopt.github.io/hyperopt/).

Note that for computational purposes, we show here a hyperparameter search for one iteration. As an example, we ran 25 iterations to obtain the results of the paper.

In [ ]:
# specify nest
nest = {0: [0], 1: [1], 2: [2, 3]}

nested_structure = {
    "mu": np.array([1, 1, 1.17]),
    "nests": nest,
    "optimise_mu": False,
}

In [ ]:
#model specification
model_specification = {
    "general_params": general_params,
    "rum_structure": rum_structure,
    "nested_logit": nested_structure,
}

#features and label column names
features = [f for f in LPMC_train.columns if f != "choice"]
label = "choice"

#create lightgbm dataset
lgb_train_set = lightgbm.Dataset(LPMC_train[features], label=LPMC_train[label], free_raw_data=False)
lgb_test_set = lightgbm.Dataset(LPMC_test[features], label=LPMC_test[label], free_raw_data=False)

In [ ]:
#specifiy seach of mu
param_space =  {'mu': hyperopt.hp.uniform('mu', 1, 2)}

#objective for hyperopt
def objective(space):

    #create mu structure
    nested_structure["mu"] = np.array([1, 1, space["mu"]])

    ce_loss = 0
    num_trees = 0

    for train_idx, test_idx in folds:
        train_set = lgb_train_set.subset(sorted(train_idx))
        test_set = lgb_train_set.subset(sorted(test_idx))

        LPMC_model_trained = rum_train(train_set, model_specification, valid_sets=[test_set])

        ce_loss += LPMC_model_trained.best_score
        num_trees += LPMC_model_trained.best_iteration


    ce_loss = ce_loss / 5
    num_trees = num_trees / 5

    return {'loss': ce_loss, 'status': hyperopt.STATUS_OK, 'best_iteration': num_trees}


#%%
#n_iter=25
n_iter = 1

trials = hyperopt.Trials()
best_classifier = hyperopt.fmin(fn=objective,
                                space=param_space,
                                algo=hyperopt.tpe.suggest,
                                max_evals=n_iter,
                                trials=trials)

print(f'Best mu: {best_classifier["mu"]} \n Best negative CE: {trials.best_trial["result"]["loss"]}')


### Cross-Validation

Once we know the optimal value of $\mu$, we can perform cross-validation to obtain the best number of trees.

Note that we use the optimal value of $\mu$ found with a bigger hyperparameter search, i.e. 1.17.

In [ ]:
_, _, folds = load_preprocess_LPMC(path="../Data/")

#optimal mu
nested_structure['mu'] = np.array([1., 1., 1.166746773143513])

ce_loss = 0
num_trees = 0

#CV with 5 folds
for i, (train_idx, test_idx) in enumerate(folds):

    #create the lightgbm CV training and validation set
    train_set = lgb_train_set.subset(sorted(train_idx))
    test_set = lgb_train_set.subset(sorted(test_idx))
    
    print('-'*50 + '\n')
    print(f'Iteration {i+1}')

    #train the model with rum_train and nest parameters
    LPMC_model_trained = rum_train(train_set, model_specification, valid_sets = [test_set])

    #aggregate results
    ce_loss += LPMC_model_trained.best_score
    num_trees += LPMC_model_trained.best_iteration
    print('-'*50 + '\n')
    print(f'Best cross entropy loss: {LPMC_model_trained.best_score}')
    print(f'Best number of trees: {LPMC_model_trained.best_iteration}')

ce_loss = ce_loss/5
num_trees = num_trees/5
print('-'*50 + '\n')
print(f'Cross validation negative cross entropy loss: {ce_loss}')
print(f'With a number of trees on average of {num_trees}')

### Testing the model on out-of-sample data

Now that we have the optimal number of trees (1243), we can train the final version of the model on the full dataset, and test it on out-of-sample data with the ```predict()``` function. Note that the dataset must be a lightgbm object in the ```predict()``` function.

In [ ]:
general_params['num_iterations'] = int(num_trees)
general_params['early_stopping_round'] = None 

LPMCnested_model_fully_trained = rum_train(lgb_train_set, model_specification)

preds = LPMCnested_model_fully_trained.predict(lgb_test_set)

ce_test = cross_entropy(preds, lgb_test_set.get_label().astype(int))

print('-'*50)
print(f'Final negative cross-entropy on the test set: {ce_test}')

### Optimising $\mu$ with `scipy.minimize`

In [ ]:
# specify nest
nest = {0: [0], 1: [1], 2: [2, 3]}

nested_structure = {
    "mu": np.array([1., 1., 1.25]),
    "nests": nest,
    "optimise_mu": [False, False, True],
    "optim_interval": 50,
}

In [ ]:
_, _, folds = load_preprocess_LPMC(path="../Data/")

#model specification
model_specification = {
    "general_params": general_params,
    "rum_structure": rum_structure,
    "nested_logit": nested_structure,
}

#features and label column names
features = [f for f in LPMC_train.columns if f != "choice"]
label = "choice"

#create lightgbm dataset
lgb_train_set = lightgbm.Dataset(LPMC_train[features], label=LPMC_train[label], free_raw_data=False)
lgb_test_set = lightgbm.Dataset(LPMC_test[features], label=LPMC_test[label], free_raw_data=False)

ce_loss = []
num_trees = 0
optimised_mu = []

for train_idx, test_idx in folds:
    train_set = lgb_train_set.subset(sorted(train_idx))
    test_set = lgb_train_set.subset(sorted(test_idx))

    LPMC_model_trained = rum_train(train_set, model_specification, valid_sets=[test_set])

    ce_loss.append(LPMC_model_trained.best_score)
    num_trees += LPMC_model_trained.best_iteration
    optimised_mu.append(LPMC_model_trained.mu)

num_trees = num_trees / 5

for i, mu in enumerate(optimised_mu):
    print(f"iteration {i} --- optimised mu: {mu}, CE: {ce_loss[i]}")


### Testing the model on out-of-sample data

Now that we have the optimal number of trees, we can train the final version of the model on the full dataset, and test it on out-of-sample data with the ```predict()``` function. Note that the dataset must be a lightgbm object in the ```predict()``` function.

In [ ]:
general_params['num_iterations'] = int(num_trees)
general_params['early_stopping_round'] = None

LPMCnested_model_fully_trained = rum_train(lgb_train_set, model_specification)

preds = LPMCnested_model_fully_trained.predict(lgb_test_set)

ce_test = cross_entropy(preds, lgb_test_set.get_label().astype(int))

print('-'*50)
print(f'Final negative cross-entropy on the test set: {ce_test}')
print(f'Opimal mu {LPMCnested_model_fully_trained.mu}')

# References

Salvadé, N., & Hillel, T. (2025). Rumboost: Gradient Boosted Random Utility Models. *Transportation Research Part C: Emerging Technologies* 170, 104897. DOI: [10.1016/j.trc.2024.104897](https://doi.org/10.1016/j.trc.2024.104897)

Hillel, T., Elshafie, M.Z.E.B., Jin, Y., 2018. Recreating passenger mode choice-sets for transport simulation: A case study of London, UK. Proceedings of the Institution of Civil Engineers - Smart Infrastructure and Construction 171, 29–42. https://doi.org/10.1680/jsmic.17.00018